## COVID Data ETL

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

dataFolder = './data/'

# Load CSV data from Netezza export
#d1 = dataFolder+'conposcovidloc.csv'
d1 = dataFolder+'conposcovidloc.csv'

In [2]:
covidDF = pd.read_csv(d1,sep=',',header=0,dtype=object)
covidDF.shape

(306997, 18)

In [3]:
covidDF.head()

,Row_ID,Accurate_Episode_Date,Case_Reported_Date,Test_Reported_Date,Specimen_Date,Age_Group,Client_Gender,Case_AcquisitionInfo,Outcome1,Outbreak_Related,Reporting_PHU_ID,Reporting_PHU,Reporting_PHU_Address,Reporting_PHU_City,Reporting_PHU_Postal_Code,Reporting_PHU_Website,Reporting_PHU_Latitude,Reporting_PHU_Longitude
0,1,2020-06-04,2020-06-06,2020-06-06,2020-06-04,50s,FEMALE,CC,Resolved,NaN,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.6474713,-79.7088933
1,2,2020-06-01,2020-06-03,2020-06-03,2020-06-02,30s,MALE,CC,Resolved,NaN,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.6474713,-79.7088933
2,3,2020-05-30,2020-06-02,2020-06-02,2020-05-30,20s,MALE,CC,Resolved,NaN,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.6474713,-79.7088933
3,4,2020-04-15,2020-04-17,2020-04-17,2020-04-16,80s,FEMALE,OB,Resolved,Yes,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.6474713,-79.7088933
4,5,2020-04-18,2020-04-21,2020-04-21,2020-04-18,20s,MALE,OB,Resolved,Yes,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.6474713,-79.7088933


In [4]:
# Convert date column to date type
t = 'Test_Reported_Date'
c = 'Case_Reported_Date'
covidDF[t]= pd.to_datetime(covidDF[t])
covidDF[c]= pd.to_datetime(covidDF[c])
covidDF.dtypes

Row_ID                               object
Accurate_Episode_Date                object
Case_Reported_Date           datetime64[ns]
Test_Reported_Date           datetime64[ns]
Specimen_Date                        object
Age_Group                            object
Client_Gender                        object
Case_AcquisitionInfo                 object
Outcome1                             object
Outbreak_Related                     object
Reporting_PHU_ID                     object
Reporting_PHU                        object
Reporting_PHU_Address                object
Reporting_PHU_City                   object
Reporting_PHU_Postal_Code            object
Reporting_PHU_Website                object
Reporting_PHU_Latitude               object
Reporting_PHU_Longitude              object
dtype: object

### Create a data frame for database import

In [11]:
#Create a new data frame

dtDFCols = ['row_id','day','month', 'day_of_week','week_in_year','weekend','holiday','season','date_case','date_add','date_test']

dtDF = pd.DataFrame(columns = dtDFCols) 
dtDF['date_case']= pd.to_datetime(dtDF['date_case'])
dtDF['date_add']= pd.to_datetime(dtDF['date_add'])
dtDF['date_test']= pd.to_datetime(dtDF['date_test'])

dtDF['date_case']=  covidDF[c]
dtDF['date_test']=  covidDF[t]

dtDF['date_add']= covidDF[c] + pd.to_timedelta(2, unit='d')

dtDF['date_test'].fillna(dtDF['date_add'], inplace = True)

dtDF['row_id'] = covidDF['Row_ID']
dtDF['day'] = dtDF['date_test'].dt.day
dtDF['month'] = dtDF['date_test'].dt.month_name()
dtDF['day_of_week'] = dtDF['date_test'].dt.day_name()
dtDF['week_in_year'] =dtDF['date_test'].dt.week

# Weekend

dtDF['weekend'] = 'no'

dtDF.loc[dtDF['date_test'].dt.day_name() == "Saturday", 'weekend'] = 'yes'
dtDF.loc[dtDF['date_test'].dt.day_name() == "Sunday", 'weekend'] = 'yes'

# Holiday

dtDF['holiday'] = 'no'

#Christmas
dtDF.loc[(dtDF['date_test'].dt.day == 25) & (dtDF['date_test'].dt.month == 12), 'holiday'] = 'yes'
#New Year
dtDF.loc[(dtDF['date_test'].dt.day == 1) & (dtDF['date_test'].dt.month == 1), 'holiday'] = 'yes'
#Canada Day
dtDF.loc[(dtDF['date_test'].dt.day == 1) & (dtDF['date_test'].dt.month == 7), 'holiday'] = 'yes'

# Map Season
seasons = {
             1: 'Winter',
             2: 'Spring',
             3: 'Summer',
             4: 'Autumn'
}
dtDF['season_id'] = dtDF['date_test'].dt.month%12 // 3 + 1
#dtDF['season_id'].fillna(covidDF[t].dt.month%12 // 3 + 1, inplace = True) 

dtDF['season'] = dtDF['season_id'].map(seasons)

dtDF.shape

del dtDF['date_case']
del dtDF['date_add']
del dtDF['date_test']
del dtDF['season_id']

dtDF.dtypes

row_id          object
day              int64
month           object
day_of_week     object
week_in_year     int64
weekend         object
holiday         object
season          object
dtype: object

In [9]:
dtDF.head()

,row_id,day,month,day_of_week,week_in_year,weekend,holiday,season
0,1,6,June,Saturday,23,yes,no,Summer
1,2,3,June,Wednesday,23,no,no,Summer
2,3,2,June,Tuesday,23,no,no,Summer
3,4,17,April,Friday,16,no,no,Spring
4,5,21,April,Tuesday,17,no,no,Spring


In [10]:
# Save Data
dtDF.to_csv(dataFolder+'Specimen Date_dimension_CSV.csv',encoding="utf-8",index=False)